<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Preparation_des_donnees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Création de données d'entrée X avec l'utilisation d'une fenêtre glissante

**1. Création d'une liste de données**

Commençons par créer un ensemble de 16 valeurs avec l'outils `Dataset` de tensorflow :

In [ ]:
import tensorflow as tf
import numpy as np

# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(16)

for valeur in dataset:
  print("%02d" %valeur.numpy())

**2. Extraction des sous-ensembles à l'aide d'une fenêtre glissante**

Utilisons maintenant l'outils `window` de la classe `Dataset` afin d'obtenir plusieurs sous-ensembles. Ces sous-ensembles sont créés à l'aide d'une fenêtre glissante appliquée sur les valeurs précédentes. Les paramètres de la fenêtre glissante sont :
 - Taille de la fenêtre : 5
 - Décalage : 1


In [ ]:
# Création d'un ensemble de 16 valeurs en 1D
dataset = tf.data.Dataset.range(16)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
dataset = dataset.window(5,shift=1)

i=0
for fenetre in dataset:
  print ("Fenêtre #%02d :" %i, end=" ")
  for valeur in fenetre:
    print ("%02d" %valeur.numpy(), end=" ")
  print()
  i = i+1

Faisons maintenant en sorte que chaque fenêtre possède le même nombre de données (5 données), en éliminant les fenêtres trop petites :

In [ ]:
# Création d'un ensemble de 16 valeurs en 1D
dataset = tf.data.Dataset.range(16)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

i=1
for fenetre in dataset:
  print ("Fenêtre #%02d :" %i, end=" ")
  for valeur in fenetre:
    print ("%02d" %valeur.numpy(),end=" ")
  print()
  i=i+1

**3. Conversion des sous-ensembles en vecteurs numpy**

Commençons par regarder comment obtenir les valeurs contenues dans chaque fenêtre sous forme de vecteur à l'aide de la méthode [batch()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch) de la classe `Dataset` de Tensorflow qui permet de regrouper les données dans des ensembles de taille spécifique :

In [ ]:
# Création d'un ensemble de 16 valeurs en 1D
dataset = tf.data.Dataset.range(16)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

for fenetre in dataset:
    vecteurs = fenetre.batch(5)
    print(list(vecteurs.as_numpy_iterator()))


Convertissons maintenant chaque vecteur au format numpy avec la fonction `.numpy()` ou bien en utilisant la fonction `asarray()` de numpy

In [ ]:
# Création d'un ensemble de 16 valeurs en 1D
dataset = tf.data.Dataset.range(16)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

for fenetre in dataset:
    vecteurs = fenetre.batch(5)
    for vecteur in vecteurs:
      print(np.asarray(vecteur))
#      print(vecteur.numpy())

Un manière plus élégante est d'utiliser la fonction d'applatissement [flat_map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#flat_map) de la classe `Dataset` de tensorflow : Cette méthode permet d'appliquer une fonction lambda à un dataset et d'applatir le résultat obtenu :

In [ ]:
# Création d'un ensemble de 16 valeurs en 1D
dataset = tf.data.Dataset.range(16)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

# Extraction des vecteurs de 5 valeurs à l'aide de la fonction flat_map
vecteurs = dataset.flat_map(lambda x: x.batch(5))

for vecteur in vecteurs:
  print(vecteur.numpy())


Nous avons maintenant nos données d'entrées X au format Numpy !

# Création des labels Y à partir des données d'entrées X

Il faut maintenant construire les labels Y à partir des données d'entrées X. Choisissons par exemple que le label associé à chaque ensemble d'entrée soit la prochaine valeur à venir de  cet ensemble :

<img src="https://github.com/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/images/X_Y2.png?raw=true" width="250">

Pour réaliser cela, on utilise la méthode [map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) de la classe `Dataset` tensorflow qui applique une fonction lambda au dataset :

In [ ]:
# Création d'un ensemble de 16 valeurs en 1D
dataset = tf.data.Dataset.range(16)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

# Extraction des vecteurs de 5 valeurs à l'aide de la fonction flat_map
vecteurs = dataset.flat_map(lambda x: x.batch(5))

# Création des ensembles X/Y à l'aide de la fonction map
data_XY = vecteurs.map(lambda x: (x[:-1],x[-1:]))

for x,y in data_XY:
  print(x.numpy(), y.numpy())

# Mélange des données X et Y

Avant d'envoyer des données dans un réseau de neurones, on a souvent besoin de les mélanger. Cela peut se faire avec la méthode [shuffle](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) de la classe `Dataset` tensorflow :

In [ ]:
# Création d'un ensemble de 16 valeurs en 1D
dataset = tf.data.Dataset.range(16)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

# Extraction des vecteurs de 5 valeurs à l'aide de la fonction flat_map
vecteurs = dataset.flat_map(lambda x: x.batch(5))

# Création des ensembles X/Y à l'aide de la fonction map
data_XY = vecteurs.map(lambda x: (x[:-1],x[-1:]))

# Mélange les données avec un buffer de  12
data_XY = data_XY.shuffle(buffer_size=12)

for x,y in data_XY:
  print(x.numpy(), y.numpy())

# Création de paires de données (X/Y)

A partir des données précédentes, on peut créer des paires d'ensembles X,Y en utlisant la méthode [batch](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch) de la classe `Dataset` :

<img src="https://github.com/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/images/Paires_X_Y_2.png?raw=true" width="700">

In [ ]:
# Création d'un ensemble de 16 valeurs en 1D
dataset = tf.data.Dataset.range(16)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

# Extraction des vecteurs de 5 valeurs à l'aide de la fonction flat_map
vecteurs = dataset.flat_map(lambda x: x.batch(5))

# Création des ensembles X/Y à l'aide de la fonction map
data_XY = vecteurs.map(lambda x: (x[:-1],x[-1:]))

# Mélange les données avec un buffer de  12
data_XY = data_XY.shuffle(buffer_size=12)

# Regroupe les données XY par paires
data_XY = data_XY.batch(2)

for x,y in data_XY:
  print("x = %s %s" %(x.numpy()[-1:],x.numpy()[:-1]))
  print("y = %s %s" %(y.numpy()[-1:],y.numpy()[:-1]))